<a href="https://colab.research.google.com/github/2303A52158/GENERATIVE-AI_2025/blob/main/Generative_AI_2303A52158_Ass_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. (1 ponto) Design a multilayer ANN architecture according to the requirements shown below. Train, test, save (.h5) and deploy the model to predict the housing price using Keras deep
learning library**

**2. (1 ponto) Calculate training and testing error metrics.**

**3. (1 ponto) Build the application by loading the saved ANN model.**

                            Tabela 1: ANN Architecture
                      Layer       Neurons Activation-Function
                  Hidden Layer-1    18           swish
                  Hidden Layer-2    26           swish
                  Hidden Layer-3    20           swish
                  Hidden Layer-4    15           swish

                            Tabela 2: Training Parameters
      loss-function epochs batch-size error-metric Optimizer
          MSE         200     64          RMSE      rmsprop

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

data = pd.read_csv('/content/Housing.csv')
categorical_features = data.select_dtypes(include=['object']).columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features))

numerical_features = data.drop(columns=categorical_features)
X = pd.concat([numerical_features.drop('price', axis=1), encoded_df], axis=1)
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(18, activation='swish', input_shape=(X_train.shape[1],)),
    Dense(26, activation='swish'),
    Dense(20, activation='swish'),
    Dense(15, activation='swish'),
    Dense(1)
])

model.compile(loss='mean_squared_error', optimizer=RMSprop(), metrics=['mae'])

history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

model.save('housing_price_model.h5')

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmae = np.sqrt(mean_absolute_error(y_train, y_train_pred))
test_rmae = np.sqrt(mean_absolute_error(y_test, y_test_pred))

print(f"Training MSE: {train_mse:.4f}")
print(f"Testing MSE: {test_mse:.4f}")
print(f"Training RMAE: {train_rmae:.4f}")
print(f"Testing RMAE: {test_rmae:.4f}")

loaded_model = load_model('housing_price_model.h5')

def predict_housing_price(features):
    user_input_df = pd.DataFrame([features[:len(numerical_features.columns) - 1]],
                                 columns=numerical_features.columns[1:])

    categorical_input_df = pd.DataFrame([features[len(numerical_features.columns) - 1:]],
                                        columns=categorical_features).astype(str)

    encoded_user_input = encoder.transform(categorical_input_df)
    encoded_user_input_df = pd.DataFrame(encoded_user_input, columns=encoder.get_feature_names_out(categorical_features))

    final_input_df = pd.concat([user_input_df, encoded_user_input_df], axis=1)

    scaled_input = scaler.transform(final_input_df)

    prediction = loaded_model.predict(scaled_input)
    return prediction[0][0]

user_input = [8000, 3, 2, 2, 1, 1, 'yes', 'no', 'no', 'yes', 'no', 'furnished']
predicted_price = predict_housing_price(user_input)
print(f"Predicted Housing Price: {predicted_price:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - loss: 25491144704000.0000 - mae: 4741285.5000 - val_loss: 30129992499200.0000 - val_mae: 5007536.5000
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 24266112237568.0000 - mae: 4635167.5000 - val_loss: 30129988304896.0000 - val_mae: 5007536.0000
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 25263746318336.0000 - mae: 4700828.0000 - val_loss: 30129988304896.0000 - val_mae: 5007536.0000
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 24991376605184.0000 - mae: 4670789.5000 - val_loss: 30129986207744.0000 - val_mae: 5007535.5000
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 24708319805440.0000 - mae: 4677204.0000 - val_loss: 30129982013440.0000 - val_mae: 5007535.0000
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25088124518400.0000 - mae: 4688573.0000 - val_loss: 30129973624832.0000 - val_mae: 5007535.0000
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 24579319791616.00

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


Training MSE: 18722288828416.0000
Testing MSE: 22606291927040.0000
Training RMAE: 2022.2636
Testing RMAE: 2090.9382
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Predicted Housing Price: 1290255.00
